In [ ]:
import sys, os
import numpy as np
import matplotlib.pyplot as plt
import scipy.io as sio
%matplotlib inline

from importlib import reload
sys.path.append(os.path.dirname(os.getcwd()) ) # add hextof-processor to path
import processor.DldFlashDataframeCreator as DldFlashProcessor
import processor.utilities as utils

In [ ]:
runs = [22137, 22133, 22135, 22138]
energies = [26.5, 29, 29.5, 31.5]
Tfrom = range(640, 720, 20)
Tto = range(780, 860, 20)
processor = DldFlashProcessor.DldFlashProcessor()
res = []
for ir, run in enumerate(runs):
    processor.runNumber = run

    ToFfrom = 600
    ToFto = 900
    ToFstep = 30*processor.TOF_STEP_TO_NS

    # delay parameters
    delayFrom = -534.6
    delayTo = -509.5
    delayStep = 0.1

    dldFrom = 0
    dldTo = 2800
    dldStep = 35

    try:
        processor.readDataframes()
    except:
        processor.readData()
        processor.storeDataframes()
    # BAM correction
    processor.postProcess()

    processor.resetBins()
    processor.addFilter('microbunchId', lb=100, ub=400)
    pX = processor.addBinning('dldPosX', dldFrom, dldTo, dldStep)
    pY = processor.addBinning('dldPosY', dldFrom, dldTo, dldStep)
    ToF = processor.addBinning('dldTime', ToFfrom, ToFto, ToFstep)
    res.append(processor.computeBinnedData())
res = np.asarray(res)

In [ ]:
res.shape

In [ ]:
plt.imshow(res[40,:,:])

In [ ]:
edc265 = res[0,:].mean(axis=(0,1))
edc290 = res[1,:].mean(axis=(0,1))
edc295 = res[2,:].mean(axis=(0,1))
edc315 = res[3,:].mean(axis=(0,1))

plt.figure(figsize=(10,4))
plt.plot(edc265, label='bias = 26.5 V')
plt.plot(edc290, label='bias = 29.0 V')
plt.plot(edc295, label='bias = 29.5 V')
plt.plot(edc315, label='bias = 31.5 V')
plt.legend()

In [ ]:
# Find the optimal shift parameter between the energy calibration scans

In [ ]:
sio.savemat('calib.mat', {'ToF':ToF, 'pX':pX, 'pY':pY, 'curves':res})